In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [2]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@726f0ba8
Data size: 1


# Prepare helper classes & functions

In [3]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [ ]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        sum += individualMatchResult.individualValues[key]!!
    }

    return sum
}

In [11]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>, avgGenIndividualRatings: MutableList<Double>, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, avgGenIndividualAvgFitnesses: MutableList<Double>){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        bestGenIndividualsRating.add(bestGenIndividualRating)
        worstGenIndividualsRating.add(worstIndividualRating)

        avgGenIndividualRating /= genProgressData.population.size
        avgGenIndividualRatings.add(avgGenIndividualRating)

        bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)

        avgGenIndividualAvgFitness /= genProgressData.population.size
        avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
    }
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [14]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualRatings = mutableListOf<Double>()

val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

val generations = mutableListOf<Int>()

for (i in 0 until bestGenIndividualsRating.size) {
    generations.add(i)
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)


<path d="M12.602872546394057 123.82124453044622 L12.602872546394057 123.82124453044622 L16.055714339926674 123.66637199459024 L19.508556133459294 117.08537057684231 L22.96139792699191 114.64971488878516 L26.414239720524527 113.05777589929357 L29.867081514057148 113.055186736726 L33.31992330758976 111.18815176282385 L36.77276510112239 111.35585842215207 L40.225606894655 109.7273056278813 L43.678448688187615 109.2628403239938 L47.13129048172024 108.9845662694513 L50.584132275252855 108.30824146927435 L54.03697406878547 107.89823488472496 L57.489815862318096 107.86903319239414 L60.94265765585071 107.0922844221206 L64.39549944938332 106.65706142487869 L67.84834124291595 108.44185241134011 L71.30118303644856 108.03830350284159 L74.75402482998118 109.70977039747262 L78.2068666235138 108.36239050446218 L81.65970841704642 104.46714772522083 L85.11255021057903 105.39857611335512 L88.56539200411166 103.30989359330022 L92.01823379764427 101.80903728150454 L95.47107559117688 99.72323330101008 L98.92391738470951 101.10877541955088 L102.37675917824212 100.98605926742636 L105.82960097177474 97.42124947649086 L109.28244276530737 98.59570377072313 L112.73528455883998 100.07589754987166 L116.18812635237259 98.16770504472373 L119.64096814590522 102.54958854372546 L123.09380993943783 99.01303108551267 L126.54665173297045 104.41627829595194 L129.99949352650307 102.77334803424708 L133.4523353200357 100.93286974542534 L136.9051771135683 101.29829704270827 L140.35801890710093 104.53495840054464 L143.81086070063355 98.71667858410123 L147.26370249416615 100.727879145124 L150.71654428769878 100.53383410265738 L154.1693860812314 100.60417301907657 L157.622227874764 100.07589754987166 L161.07506966829663 99.46194618362662 L164.52791146182926 97.33021756668839 L167.98075325536186 96.88716108348223 L171.4335950488945 97.95948052960756 L174.88643684242712 101.17725623106867 L178.33927863595972 97.45372569795123 L181.79212042949234 99.47569412918153 L185.24496222302497 100.10596229639165 L188.69780401655757 98.70714945049465 L192.1506458100902 98.57381773184306 L195.60348760362282 97.79687604357429 L199.05632939715542 97.69510672430042 L202.50917119068805 96.58846818218129 L205.96201298422068 147.47536160643375 L209.41485477775328 154.41237287722367 L212.8676965712859 163.73113956356102 L216.32053836481853 172.74097854127146 L219.77338015835113 191.113803040542 L223.22622195188376 192.181335074312 L226.67906374541639 196.60111680579797 L230.13190553894898 200.43171682626883 L233.5847473324816 209.0454545454545 L237.03758912601424 206.56299593639636 L240.49043091954684 201.4116031687316 L243.94327271307947 202.2134312783852 L247.3961145066121 203.5902667031939 L250.8489563001447 203.28557303688336 L254.30179809367732 202.6339163561493 L257.75463988721 198.17636331191272 L261.20748168074255 200.00199706868034 L264.6603234742752 200.00199706868034 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M12.602872546394057 58.10592262788967 L12.602872546394057 58.10592262788967 L16.055714339926674 56.00250726514632 L19.508556133459294 73.0431517787149 L22.96139792699191 54.66094029593954 L26.414239720524527 69.64807936284686 L29.867081514057148 58.617703608502325 L33.31992330758976 37.65136065639257 L36.77276510112239 56.13567144974941 L40.225606894655 55.15834888590746 L43.678448688187615 42.12287994812601 L47.13129048172024 64.77634661321665 L50.584132275252855 55.78747985230359 L54.03697406878547 52.719484666982254 L57.489815862318096 56.12887362881219 L60.94265765585071 62.43367739886099 L64.39549944938332 50.933729090544674 L67.84834124291595 48.96928575941877 L71.30118303644856 59.640001449179906 L74.75402482998118 61.67245345041424 L78.2068666235138 56.403395936026385 L81.65970841704642 64.1752446008119 L85.11255021057903 63.70312857537668 L88.56539200411166 58.587989160447336 L92.01823379764427 43.961048297794335 L95.47107559117688 49.41784040548902 L98.92391738470951 69.18571077296018 L102.37675917824

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [27]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 30, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 45, 0))
selectedIndividuals.add(SelectedIndividual(0, 0, 70, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val (bestGenIndividuals, worstGenIndividuals, _) = prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividuals[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividuals[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys alphabetically
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            continue
        }
        dataset.add(-individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    datasets.add(dataset)
}

// TODO Check if correct individuals are selected for visualization !!
// TODO Make this more generic (e.g., for more than 2 individuals)
var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

<path d="M170.5 126.66161323905703 L179.05244498985994 127.50395554824331 L187.27622431502596 129.99861173325104 L194.85530274798052 134.04971355590973 L201.49842055494133 139.50157944505867 L206.95028644409027 146.14469725201948 L211.00138826674896 153.72377568497404 L213.4960444517567 161.94755501014006 L214.33838676094297 170.5 L213.4960444517567 179.05244498985994 L211.00138826674896 187.27622431502596 L206.9502864440903 194.85530274798052 L201.49842055494133 201.49842055494133 L194.85530274798052 206.9502864440903 L187.27622431502596 211.00138826674896 L179.05244498985994 213.4960444517567 L170.5 214.33838676094297 L161.94755501014006 213.4960444517567 L153.72377568497404 211.00138826674896 L146.14469725201948 206.9502864440903 L139.50157944505867 201.49842055494133 L134.0497135559097 194.85530274798052 L129.99861173325104 187.27622431502598 L127.5039555482433 179.05244498985996 L126.66161323905703 170.5 L127.5039555482433 161.94755501014006 L129.99861173325104 153.72377568497402 L134.0497135559097 146.14469725201948 L139.50157944505867 139.50157944505867 L146.14469725201948 134.04971355590973 L153.72377568497402 129.99861173325104 L161.94755501014004 127.50395554824331 L170.5 126.66161323905703 " stroke="rgb(233,233,233)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M170.5 82.82322647811407 L179.0938266139378 83.24541410712186 L187.60488997971987 84.50791109648662 L195.95122390451326 86.59855891004963 L204.0524486300519 89.49722346650208 L211.83054493364907 93.175989041594 L219.21060549596106 97.59942711181947 L226.12155629942137 102.72493755046494 L232.49684110988267 108.50315889011733 L238.27506244953506 114.87844370057863 L243.40057288818053 121.78939450403897 L247.824010958406 129.16945506635093 L251.50277653349792 136.9475513699481 L254.40144108995037 145.04877609548677 L256.4920889035134 153.39511002028013 L257.75458589287814 161.9061733860622 L258.17677352188593 170.5 L257.75458589287814 179.0938266139378 L256.4920889035134 187.60488997971987 L254.40144108995037 195.95122390451323 L251.50277653349792 204.0524486300519 L247.824010958406 211.83054493364907 L243.40057288818056 219.21060549596103 L238.27506244953506 226.12155629942134 L232.49684110988267 232.49684110988267 L226.12155629942137 238.27506244953506 L219.21060549596103 243.40057288818056 L211.83054493364907 247.824010958406 L204.0524486300519 251.50277653349792 L195.95122390451326 254.40144108995037 L187.6048899797199 256.4920889035134 L179.0938266139378 257.75458589287814 L170.5 258.17677352188593 L161.9061733860622 257.75458589287814 L153.39511002028013 256.4920889035134 L145.04877609548677 254.40144108995037 L136.9475513699481 251.50277653349792 L129.16945506635093 247.824010958406 L121.78939450403897 243.40057288818056 L114.87844370057866 238.27506244953506 L108.50315889011733 232.49684110988267 L102.72493755046499 226.1215562994214 L97.59942711181944 219.21060549596103 L93.175989041594 211.83054493364907 L89.49722346650209 204.05244863005197 L86.59855891004965 195.95122390451326 L84.5079110964866 187.6048899797199 L83.24541410712185 179.09382661393778 L82.82322647811408 170.5 L83.24541410712185 161.90617338606225 L84.5079110964866 153.39511002028013 L86.59855891004965 145.04877609548677 L89.49722346650208 136.94755136994806 L93.175989041594 129.16945506635093 L97.59942711181944 121.78939450403897 L102.72493755046496 114.87844370057863 L108.5031588901173 108.50315889011733 L114.8784437005786 102.72493755046497 L121.78939450403895 97.59942711181947 L129.16945506635093 93.175989041594 L136.94755136994803 89.49722346650208 L145.04877609548674 86.59855891004963 L153.3951100202801 84.50791109648662 L161.90617338606222 83.24541410712186 L170.49999999999997 82.82322647811407 " stroke="rgb(233,233,233)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M170.5 38.9848397171711 L183.39073992090673 39.61812116068279 L196.1573349695798 41.51186664472988 L208.67683585676986 44.64783836507445 L220.8286729450779 48.99583519975312 L2

# Individual node counts

In [32]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 30, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 45, 0))
selectedIndividuals.add(SelectedIndividual(0, 0, 70, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val (bestGenIndividuals, worstGenIndividuals, _) = prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividuals[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividuals[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 400 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)
    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

Conf: 0 Run: 0 Gen: 30 IndividualID: 19 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Conf: 0 Run: 0 Gen: 45 IndividualID: 19 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Conf: 0 Run: 0 Gen: 70 IndividualID: 19 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector

# Convergence graphs

In [34]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData

val n = 3
val Generation = mutableListOf<Int>()
val Rating = mutableListOf<Double>()

val convergenceDataGraph = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;
for (i in 0 until convergenceDataGraph.size) {
    Generation.addAll(List(n) {i + 1})
    val rating = Math.abs(convergenceDataGraph[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
}

val data = mapOf<String, Any>(
    "Generation" to Generation,
    "Rating" to Rating
)

letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Convergence graph (Tournament between generation best individuals)")

0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 Convergence graph (Tournament between generation best individuals) 
 
 
 
 
 Rating 
 
 
 
 
 Generation